# this notebook is for:

1) importing the recall accuracy and confmat scores   
2) creating an accuracy brain map via accuracy and confmat forms (although confmat superior here) 
3a) doing this brainmap for the classification of rooms on room eve nts  
3b) doing this brainmap for classification of objects on object events   
3c) doing this brainmap for the classification of rooms on object events  (room reinstatement on objects)
---> note that 3c means importing recall classificaiton results from 20250524 while 3a and 3b can come from date = 20240401 which is an earlier date and that's fine.  

# import

In [1]:
from tqdm import tqdm_notebook
import deepdish as dd
import numpy as np
import os
import h5py
import pandas as pd
from scipy import stats
from _searchlight_util import *
import matplotlib.pyplot as plt
from _classification_util import Convert_SourceItem_to_PairedItem_Order
from _mempal_util import room_obj_assignments, save_obj, load_obj, create_dirs, subject_ids

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']
roi = 'SL'
nSubj =25

In [2]:
def GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath = '../PythonData2024/Output/ClassifyRecalls'):
    '''
    - grabs the whole-brain results of classifying recalls (GRs+FRs)
    - returns 'scores' which is a dict with the data in measure_list
    
    '''

    trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']

    nPerm = 1000
    nTrials = len(trials)
    nItems   = 23

    
    scores = {}

    for betatype in tqdm_notebook(betatypes):
        scores[betatype] = {}

        print("...BETATYPE: ", betatype)

        for measuretype in tqdm_notebook(measure_list):
            scores[betatype][measuretype] = {}

            print("............MEASURE: ", measuretype)

            for hem in ['R','L']:
                print("...............HEM: ",hem)

                hem_label = hem
                betatype_label=betatype

                ###
                ### Specific initiation for different array shapes
                ###

                if 'RIP_window' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),win_size,nTrials,nSubj))) 

                if 'evidence_window' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),win_size,nItems, nTrials,nSubj))) 

                if 'accuracy' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),nTrials,nSubj,nPerm+1)))

    #             if 'real_minus_prior' == measuretype:
    #                 scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),nTrials,nSubj)))   

                if 'evidence' == measuretype:
                    scores[betatype][measuretype][hem] = {}

                if 'conf_mat' == measuretype: # comes in as (nItems,nItems,nTrials,nSubj,nPerm+1)
                    scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems,nTrials,nSubj), fill_value=np.nan)
#                     scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems), fill_value=np.nan)


                if 'RIPs' == measuretype:
    #                 scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems,nTrials,nSubj), fill_value=np.nan)
                    scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nTrials,nSubj), fill_value=np.nan)
                    


                if 'avg_RIPs' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),2, nItems,nSubj))) # for GR and FR
                ###
                ### Load the saved .h5 files
                ###

                for roi_id in tqdm_notebook(range(len(SLlist[hem]))[:]):

#                     dirpath = '../PythonData2024/Output/ClassifyRecalls'

                    fname = '{}_{}{:03d}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls'.format(
                    date,
                    roi,
                    roi_id,
                    hem_label,
                    betatype_label,
                    win_size,
                    shift) + '.h5'

                    fullpath = os.path.join(dirpath,fname)

                    temp = dd.io.load(fullpath,'/{}/{}/{}'.format(betatype,hem,measuretype))

                    if measuretype=='conf_mat':
#                         print('.......conf_mat shape: ', temp.shape)
#                         scores[betatype][measuretype][hem][roi_id] = np.nanmean(np.nanmean(temp[:,:,:,:,0],2),2)
                        scores[betatype][measuretype][hem][roi_id,:,:,:,:] = temp[:,:,:,:,0] # permutations are empty, so no need to collect them
#                          scores[betatype][measuretype][hem][roi_id,:,:] = temp[:,:,:,:,0].sum(-1).sum(-1) # permutations are empty, so no need to collect them

                        
                    elif measuretype == "avg_RIPs":
                        scores[betatype][measuretype][hem][roi_id,0,:,:] = temp['GR']
                        scores[betatype][measuretype][hem][roi_id,1,:,:] = temp['FR']
                    else:
                        scores[betatype][measuretype][hem][roi_id] = temp
                        
                        
    return scores

                


# confmat and accuracy for room recall during room events and object recall during object events (for GRs and FRs)

3a and 3b

## confmat

run separately, since it takes a long time.

In [11]:
## 
## IMPORT -- when rooms classify room events and objects classify object events
##

#accuracy[betatype][hem]= np.full((nTrials,nSubj,nPerm+1),fill_value=np.nan)
# conf_mat[betatype][hem]= np.full((nItems,nItems,nTrials,nSubj,nPerm+1), fill_value=np.nan)
# evidence[betatype][hem]= {} #dict of trials and subjects
# cp_window[betatype][hem]= np.full((win_size,nTrials,nSubj), fill_value=np.nan) 
# trial_length[betatype][hem] = np.full((nTrials,nSubj),fill_value=np.nan)

dirpath = '../PythonData2024/Output/ClassifyRecalls'


date = 20240401; shift = 4; win_size=9 ;


# betatypes = ['rooms','objects']
betatypes = ['objects', 'rooms'] #['objects']
roi = 'SL'
measure_list = [
#     'conf_mat', #  # only acquire if doing the accurazy brainmap not necessary for the ROCN evidence
#     "avg_RIPs", 
#     'evidence',
    'accuracy', #
#     'RIP_window', # 
#     'RIPs',
#     'conf_mat', # 
  ]

scores = GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath)

     

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2 [00:00<?, ?it/s]

...BETATYPE:  objects


/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

............MEASURE:  accuracy
...............HEM:  R


  0%|          | 0/1483 [00:00<?, ?it/s]

...............HEM:  L


  0%|          | 0/1483 [00:00<?, ?it/s]

In [ ]:
# ## SAVE CONFMAT since it takes a long time to load up

# date = 20240401; shift = 4; win_size=9 ;

# fname = '{}_{}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls_CONFMAT'.format(
# date,
# roi,
# 'both',
# 'both',
# win_size,
# shift) + '.pkl'

# conf_mat_scores = {}
# for betatype in tqdm_notebook(betatypes):
#     conf_mat_scores[betatype] = {}
#     for hem in ['R','L']:
#         conf_mat_scores[betatype][hem] = scores[betatype]['conf_mat'][hem].copy()
        
# save_obj("../PythonData2024/Output/confmats/{}".format(fname),conf_mat_scores)  

# print("...confmats saved.")
    


### load confmat

In [ ]:
## LOAD AFTER TAKING forever to make confmat array

date = 20240401; shift = 4; win_size=9 ;

fname = '{}_{}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls_CONFMAT'.format(
date,
roi,
'both',
'both',
win_size,
shift) + '.pkl'

        
conf_mat_scores = load_obj("../PythonData2024/Output/confmats/{}".format(fname))  

# print("...confmats saved.")
    


In [8]:
conf_mat_scores.keys(), conf_mat_scores['objects']['L'].shape, conf_mat_scores['rooms']['L'].shape

(dict_keys(['objects', 'rooms']),
 (1483, 23, 23, 12, 25),
 (1483, 23, 23, 12, 25))

In [ ]:

#scores --> nPerm+1
#ev_prank --> nPerm+1

nv = 40962 #verts in fsaverage6 brain
# raw_vox = {} # raw values
# p_vox = {} # non-parametric p-values
# q_vox = {} # q-values from FDR-correction
# z_vox = {} # z-values

#     'cp_window', # (nSLs, 9,11,25)
#     'evidence_window', # (nSLs, 9,23,11,25)
#     'accuracy', #(nSLs,11,25,1001)
#     'real_minus_prior', #(nSLs,11,25)

chance = 1/23

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']
nPerm = 1000

# trial_to_use = 'GR'
# trials_to_use
measures = {}
for trial_to_use in ['GR','FR']:
    
    ## which type of recall are we going to get accuracy from?
    ## choose between guided recalls (GR) or free recalls (FR)
    trials_to_use = np.arange(len(trials))[:-1] if trial_to_use == 'GR' else [np.arange(len(trials))[-1]]

    measures[trial_to_use] = {}
    for betatype in ['objects','rooms']:# ['rooms','objects']:
        measures[trial_to_use][betatype] = {}
        
        for measuretype in ['accuracy','conf_mat']: #measure_list:
            measures[trial_to_use][betatype][measuretype] = {}

            if 'accuracy' == measuretype:

                print("....first if: {} {}".format(betatype, measuretype))

                dd_hem = {"L":np.nanmean(np.nanmean(scores[betatype][measuretype]['L'][:,trials_to_use,:,:],1),1),
                          "R": np.nanmean(np.nanmean(scores[betatype][measuretype]['R'][:,trials_to_use,:,:],1),1)} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}

                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)
                

            ### SHOULD REMOVE THIS, 20230812 DONT NEED IT
            if 'RIPs' == measuretype or "avg_RIPs" == measuretype:
                t_vox = {}
                t_p_vox = {}
                raw_vox = {}

                if 'RIPs'==measuretype:
                    dd_hem = {"L":np.nanmean(np.nanmean(scores[betatype][measuretype]['L'][:,:,trials_to_use,:],1),1),
                          "R": np.nanmean(np.nanmean(scores[betatype][measuretype]['R'][:,:,trials_to_use,:],1),1)} #shape (nSLs,nSubj)

                elif "avg_RIPs" == measuretype:
                    idx2use = 0 if trial_to_use=='GR' else 1
                    #scores[betatype]['avg_RIPs']['R'].shape #(1483, 2, 23, 25)
                    dd_hem = {"L": np.nanmean(scores[betatype][measuretype]['L'][:,idx2use,:,:],1),
                             "R": np.nanmean(scores[betatype][measuretype]['R'][:,idx2use,:,:],1)} #shape (nSLs,nSubj)
                
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                for hem in ['L','R']:
                    t_vox[hem] = np.full((nv),fill_value=np.nan)
                    t_p_vox[hem] = np.full((nv),fill_value=np.nan)
                    raw_vox[hem] = np.nanmean(dd_vox[hem],1) #avg out subj
                    for vi in tqdm_notebook(range(nv)):

                        # get t value from 25 subj numbers for every vertex
    #                     t, p = stats.ttest_1samp(dd_vox[hem][vi,:],chance,alternative='greater') #one sided t-test 
                        t, p = stats.ttest_1samp(dd_vox[hem][vi],chance) #only two-sided available
                        p = p / 2 # make it 1 sided
                        t_vox[hem][vi] = t
                        t_p_vox[hem][vi] = p

                measures[trial_to_use][betatype][measuretype]['t_q'] = FDR_p_hem(t_p_vox)
                measures[trial_to_use][betatype][measuretype]['t'] = t_vox
                measures[trial_to_use][betatype][measuretype]['raw'] = raw_vox
                
            if 'conf_mat' == measuretype: #accuracy based on confmat
                
                confmat_acc_perm = {}
                for hem in ['L','R']:
                    confmat_acc_perm[hem] = np.full((len(SLlist[hem]), nPerm+1), fill_value=np.nan)
                    confmat = scores[betatype]['conf_mat'][hem][:,:,:,trials_to_use,:].sum(-1).sum(-1) # shape (nSLs, 23,23)

                    ## PERMUTATION:
                    print('...runing confmat permutation.')
                    for sl in tqdm_notebook(range(len(SLlist[hem]))):
                        for p in range(nPerm+1):
                            if p > 0:
                                permconf = confmat[sl,np.random.permutation(np.arange(23)),:].copy()
                            else:
                                permconf = confmat[sl].copy()

                            confmat_acc_perm[hem][sl,p] = np.diag(permconf).sum() / permconf.sum()
                            
                    
                dd_hem = {"L":confmat_acc_perm['L'],"R": confmat_acc_perm['R']} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}
                
                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)

                


###
# ### SAVE SAVE SAVE
# ###
# date = 20230814 # after re-running classifyrecalls on objects
# date = 20240108 # 
date = 20240401 # second permutation fix, keep structure intact

dirpath = '../PythonData2024/Output/brainmaps/'
fname = '{}_ClassifyRecalls_Accuracy_shift{}'.format(date,shift) + '.pkl'
fullpath = os.path.join(dirpath,fname)

#     data = measures
save_obj(fullpath,measures)
print('...saving in: ',fullpath)


print('complete.')

# confmat and accuracy for room recall during object events (for GRs and FRs)

3c

In [ ]:
###
### IMPORT --- when rooms classify object events [revision update]
###

dirpath = '../PythonData2024/Output/ClassifyRecalls_RoomsOnObjectEvents/'


date = 20250524; shift = 4; win_size=9 ;
betatypes    = ['rooms']
roi          = 'SL'
measure_list = [
    'conf_mat', #  # only acquire if doing the accurazy brainmap not necessary for the RRCN evidence
#     "avg_RIPs", 
#     'evidence',
#     'accuracy', #
#     'RIP_window', # 
#     'RIPs',
#     'conf_mat', # 
  ]

scores2 = GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath)


In [ ]:
# ## SAVE CONFMAT since it takes a long time to load up

# # date = 20240401; shift = 4; win_size=9 ;

# fname = '{}_{}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls_RoomsOnObjectEvents_CONFMAT'.format(
# date,
# roi,
# 'both',
# 'both',
# win_size,
# shift) + '.pkl'

# conf_mat_scores = {}
# for betatype in tqdm_notebook(betatypes):
#     conf_mat_scores[betatype] = {}
#     for hem in ['R','L']:
#         conf_mat_scores[betatype][hem] = scores2[betatype]['conf_mat'][hem].copy()
        
# save_obj("../PythonData2024/Output/confmats/{}".format(fname),conf_mat_scores)  

# print("...confmats saved.")
    


In [ ]:
# load
# date = 20240401; shift = 4; win_size=9 ;

fname = '{}_{}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls_RoomsOnObjectEvents_CONFMAT'.format(
date,
roi,
'both',
'both',
win_size,
shift) + '.pkl'
        
scores2 = load_obj("../PythonData2024/Output/confmats/{}".format(fname))  


In [ ]:
###
### IMPORT --- when rooms classify object events [revision update]
###

dirpath = '../PythonData2024/Output/ClassifyRecalls_RoomsOnObjectEvents/'


date = 20250524; shift = 4; win_size=9 ;
betatypes    = ['rooms']
roi          = 'SL'
measure_list = [
#     'conf_mat', #  # only acquire if doing the accurazy brainmap not necessary for the RRCN evidence
    "avg_RIPs", 
#     'evidence',
    'accuracy', #
#     'RIP_window', # 
#     'RIPs',
#     'conf_mat', # 
  ]

scores = GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath)


In [ ]:

#scores2 --> nPerm+1
#ev_prank --> nPerm+1

nv = 40962 #verts in fsaverage6 brain
# raw_vox = {} # raw values
# p_vox = {} # non-parametric p-values
# q_vox = {} # q-values from FDR-correction
# z_vox = {} # z-values

#     'cp_window', # (nSLs, 9,11,25)
#     'evidence_window', # (nSLs, 9,23,11,25)
#     'accuracy', #(nSLs,11,25,1001)
#     'real_minus_prior', #(nSLs,11,25)

chance = 1/23

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']
nPerm = 1000

# trial_to_use = 'GR'
# trials_to_use
measures = {}
for trial_to_use in ['GR','FR']:
    
    ## which type of recall are we going to get accuracy from?
    ## choose between guided recalls (GR) or free recalls (FR)
    trials_to_use = np.arange(len(trials))[:-1] if trial_to_use == 'GR' else [np.arange(len(trials))[-1]]

    measures[trial_to_use] = {}
    for betatype in ['rooms']:# ['rooms','objects']:
        measures[trial_to_use][betatype] = {}
        
        for measuretype in ['conf_mat']: #measure_list:
            measures[trial_to_use][betatype][measuretype] = {}

            if 'accuracy' == measuretype:

                print("....first if: {} {}".format(betatype, measuretype))

                dd_hem = {"L":np.nanmean(np.nanmean(scores2[betatype][measuretype]['L'][:,trials_to_use,:,:],1),1),
                          "R": np.nanmean(np.nanmean(scores2[betatype][measuretype]['R'][:,trials_to_use,:,:],1),1)} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}

                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)
                

            ### SHOULD REMOVE THIS, 20230812 DONT NEED IT
            if 'RIPs' == measuretype or "avg_RIPs" == measuretype:
                t_vox = {}
                t_p_vox = {}
                raw_vox = {}

                if 'RIPs'==measuretype:
                    dd_hem = {"L":np.nanmean(np.nanmean(scores2[betatype][measuretype]['L'][:,:,trials_to_use,:],1),1),
                          "R": np.nanmean(np.nanmean(scores2[betatype][measuretype]['R'][:,:,trials_to_use,:],1),1)} #shape (nSLs,nSubj)

                elif "avg_RIPs" == measuretype:
                    idx2use = 0 if trial_to_use=='GR' else 1
                    #scores2[betatype]['avg_RIPs']['R'].shape #(1483, 2, 23, 25)
                    dd_hem = {"L": np.nanmean(scores2[betatype][measuretype]['L'][:,idx2use,:,:],1),
                             "R": np.nanmean(scores2[betatype][measuretype]['R'][:,idx2use,:,:],1)} #shape (nSLs,nSubj)
                
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                for hem in ['L','R']:
                    t_vox[hem] = np.full((nv),fill_value=np.nan)
                    t_p_vox[hem] = np.full((nv),fill_value=np.nan)
                    raw_vox[hem] = np.nanmean(dd_vox[hem],1) #avg out subj
                    for vi in tqdm_notebook(range(nv)):

                        # get t value from 25 subj numbers for every vertex
    #                     t, p = stats.ttest_1samp(dd_vox[hem][vi,:],chance,alternative='greater') #one sided t-test 
                        t, p = stats.ttest_1samp(dd_vox[hem][vi],chance) #only two-sided available
                        p = p / 2 # make it 1 sided
                        t_vox[hem][vi] = t
                        t_p_vox[hem][vi] = p

                measures[trial_to_use][betatype][measuretype]['t_q'] = FDR_p_hem(t_p_vox)
                measures[trial_to_use][betatype][measuretype]['t'] = t_vox
                measures[trial_to_use][betatype][measuretype]['raw'] = raw_vox
                
            if 'conf_mat' == measuretype: #accuracy based on confmat
                
                confmat_acc_perm = {}
                for hem in ['L','R']:
                    confmat_acc_perm[hem] = np.full((len(SLlist[hem]), nPerm+1), fill_value=np.nan)
                    confmat = scores2[betatype]['conf_mat'][hem][:,:,:,trials_to_use,:].sum(-1).sum(-1) # shape (nSLs, 23,23)

                    ## PERMUTATION:
                    print('...runing confmat permutation.')
                    for sl in tqdm_notebook(range(len(SLlist[hem]))):
                        for p in range(nPerm+1):
                            if p > 0:
                                permconf = confmat[sl,np.random.permutation(np.arange(23)),:].copy()
                            else:
                                permconf = confmat[sl].copy()

                            confmat_acc_perm[hem][sl,p] = np.diag(permconf).sum() / permconf.sum()
                            
                    
                dd_hem = {"L":confmat_acc_perm['L'],"R": confmat_acc_perm['R']} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}
                
                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)

                


###
### SAVE SAVE SAVE
###
# date = 20230814 # after re-running classifyrecalls on objects
# date = 20240108 # 
# date = 20240401 # second permutation fix, keep structure intact

dirpath = '../PythonData2024/Output/brainmaps/'
fname = '{}_ClassifyRecalls_ConfMatAccuracy_RoomsOnObjectEvents_shift{}'.format(date,shift) + '.pkl'
fullpath = os.path.join(dirpath,fname)

#     data = measures
save_obj(fullpath,measures)
print('...saving in: ',fullpath)


print('complete.')